# Profiling FLASH

In [1]:
import datetime
import itertools
import logging
import os
import pathlib
import pprint
import re
import shutil
import subprocess
import sys
import typing as t

import git
import pandas as pd

In [2]:
logging.basicConfig(level=logging.WARNING)

_LOG = logging.getLogger('profile_flash')

In [3]:
try:
    _HERE = pathlib.Path(__file__).parent.resolve()
except NameError:
    _HERE = pathlib.Path(os.getcwd()).resolve()

_RESULTS_ROOT = pathlib.Path(_HERE, 'results')

_HERE, _RESULTS_ROOT

(PosixPath('/nfs2/mbysiek/Projects/docker-transpyle-flash'),
 PosixPath('/nfs2/mbysiek/Projects/docker-transpyle-flash/results'))

check if the envvars are properly set

In [4]:
os.environ['CPATH']

'/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/superlu-5.2.1-bz4ai7ydn6puimqffcglkl7ky5mwsmq4/include:/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/hpctoolkit-master-wxx6jn7xuoo7lzqhg7kwxx4xl2nrlooc/include:/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/papi-5.6.0-5p5d3qi2imn6m3szyl47gj73dpixhxpf/include:/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/gcc-8.2.0-35mxytud35rkqxpuwxalsxp5s3gqq23f/include:/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/mpc-1.1.0-lfad2lpghnhcljoohbmyz4iznbldjxte/include:/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/mpfr-4.0.1-baptjiihukqeehfh7a3ecoualvmw4jtu/include:/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/isl-0.19-j2nkr2ed5lm3cngy5zamecra6vw6fyks/include:/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/gmp-6.1.2-2ii23bc4vfcihjejzkzw5k53sr7

In [5]:
os.environ['LD_LIBRARY_PATH']

'/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/superlu-5.2.1-bz4ai7ydn6puimqffcglkl7ky5mwsmq4/lib:/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/hpctoolkit-master-wxx6jn7xuoo7lzqhg7kwxx4xl2nrlooc/lib:/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/papi-5.6.0-5p5d3qi2imn6m3szyl47gj73dpixhxpf/lib:/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/gcc-8.2.0-35mxytud35rkqxpuwxalsxp5s3gqq23f/lib64:/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/gcc-8.2.0-35mxytud35rkqxpuwxalsxp5s3gqq23f/lib:/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/mpc-1.1.0-lfad2lpghnhcljoohbmyz4iznbldjxte/lib:/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/mpfr-4.0.1-baptjiihukqeehfh7a3ecoualvmw4jtu/lib:/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/isl-0.19-j2nkr2ed5lm3cngy5zamecra6vw6fyks/lib:/nfs2/mbysiek/So

In [6]:
shutil.which('hpcrun')

'/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/hpctoolkit-master-wxx6jn7xuoo7lzqhg7kwxx4xl2nrlooc/bin/hpcrun'

get profiling framework implementation

In [7]:
import profiling_flash
from profiling_flash import profile_experiment, date_str, profile_path, profile_db_path

In [8]:
import hpctoolkit_dataframe
from hpctoolkit_dataframe import HPCtoolkitDataFrame

# run the experiments

In [9]:
_NOW = datetime.datetime.now()

In [10]:
_NOW

datetime.datetime(2018, 9, 12, 17, 17, 24, 213713)

In [11]:
profiling_flash._NOW = _NOW

In [ ]:
clean_flash(pathlib.Path(_HERE, 'flash-subset', 'FLASH4.4', 'object'), 'subset_sedov_outlined_a')

In [ ]:
# Experiment 1

branch_nicknames = {
    'transpyle_experiments': 'base',
    'transpyle_experiments_outlined': 'outlined',
    'transpyle_experiments_autoinlined': 'autoinlined'}
problems = ['Sod', 'Sedov']
samples = 50
events = None

test_name_template = 'subset_{}_{}_a'

In [ ]:
# Experiment 2

branch_nicknames = {
    'transpyle_experiments': 'base',
    #'transpyle_experiments_outlined': 'outlined',
    #'transpyle_experiments_autoinlined': 'autoinlined'
    }
problems = ['Sod', 'Sedov']
samples = 3
events = {'PAPI_MEM_WCY': 100}

test_name_template = 'subset_{}_{}_papi'

In [ ]:
for branch, nickname in branch_nicknames.items():
    for problem in problems:
        test_name = test_name_template.format(problem, nickname)
        profile_experiment('flash-subset', '{} -auto -2d +Mode1'.format(problem), branch, samples,
                           events=events,
                           test_name=test_name)

In [ ]:
# Experiment 3

branch_nicknames = {
    'transpyle_experiments': 'base',
    #'transpyle_experiments_outlined': 'outlined',
    #'transpyle_experiments_autoinlined': 'autoinlined'
    }
problems = ['Sod', #'Sedov'
           ]
samples = 1
events = {'PAPI_TOT_CYC': 100000, 'WALLCLOCK': 1 / 10}
mpi_processes = [1, 2, 4]

test_name_template = 'subset_{}_{}_scale_{}'

In [ ]:
for (branch, nickname), problem, mpi_proc in itertools.product(branch_nicknames.items(), problems, mpi_processes):
    test_name = test_name_template.format(problem, nickname, mpi_proc)
    profile_experiment('flash-subset', '{} -auto -2d +Mode1'.format(problem), branch, samples,
                       events=events, mpi_proc=mpi_proc, rebuild=False,
                       test_name=test_name)

In [12]:
# Experiment 4

branch_nicknames = {
    'SNIa_spack': 'base'}
problems = [
    ('SNIa_DoubleDetonation', '-auto -3d +cartesian -nxb=16 -nyb=16 -nzb=16 -maxblocks=100 xnet=True xnetData=Data_SN160 xnetGPU=False +uhd +starkiller starkillerGPU=False +Mode1 -debug +newMpole -objdir=sn1a-doubledet +noio'),
    #('SNIa_DoubleDetonation', '-auto -3d +cartesian -nxb=16 -nyb=16 -nzb=16 -maxblocks=100 xnet=True xnetData=Data_SN160 xnetGPU=False +uhd +starkiller starkillerGPU=False +Mode1 -debug +newMpole -objdir=sn1a-doubledet'),
    #('SNIa_DoubleDetonation', '-auto -3d +cartesian -nxb=16 -nyb=16 -nzb=16 -maxblocks=100 xnet=True xnetData=Data_SN160 xnetGPU=False +uhd +starkiller starkillerGPU=False +Mode1 +newMpole -objdir=sn1a-doubledet +noio'),
    #('SNIa_DoubleDetonation', '-auto -3d +cartesian -nxb=16 -nyb=16 -nzb=16 -maxblocks=100 xnet=True xnetData=Data_SN160 xnetGPU=False +uhd +starkiller starkillerGPU=False +Mode1 +newMpole -objdir=sn1a-doubledet')
    ]
samples = 1
events = {'PAPI_TOT_CYC': 100000, 'WALLCLOCK': 1 / 10}
mpi_processes = [0] #[0, 1, 2, 4]

test_name_template = 'subset_{}_{}_{}'

In [13]:
for (branch, nickname), (problem, options), mpi_proc in itertools.product(branch_nicknames.items(), problems, mpi_processes):
    test_name = test_name_template.format(problem, nickname, mpi_proc)
    profile_experiment('flash-subset', '{} {}'.format(problem, options), branch, samples,
                       events=events, mpi_proc=mpi_proc, rebuild=False,
                       test_name=test_name)

### review past experiments

In [ ]:
# Experiment 1

test_date = datetime.datetime(2018, 9, 5, 12, 5, 19, 934255)

In [ ]:
# Experiment 2

test_date = datetime.datetime(2018, 9, 11, 10, 2, 31, 884551)

In [ ]:
# Experiment 3

test_date = datetime.datetime(2018, 9, 11, 11, 37, 45, 878931)

In [14]:
# Experiment 4

test_date = datetime.datetime(2018, 9, 12, 17, 17, 24, 213713)

# compare bottlenecks

experiment 1

In [11]:
%%time

timings = {}
hot_paths = {}

for (branch, nickname), problem in itertools.product(branch_nicknames.items(), problems):
    test_name = test_name_template.format(problem, nickname)
    _profile_path = profile_db_path(test_date, test_name=test_name).joinpath('experiment.xml')
    df = HPCtoolkitDataFrame(path=_profile_path, max_depth=None)
    timings[nickname, problem] = df
    hot_paths[nickname, problem] = df.hot_path()

CPU times: user 36.8 s, sys: 486 ms, total: 37.3 s
Wall time: 37.3 s


In [51]:
hot_paths['base', 'Sedov'].compact[-3:]

,CPUTIME (usec):Mean (I),CPUTIME (usec):Mean (I) ratio of total,CPUTIME (usec):Mean (I) ratio of parent
"('<program root>.6', 'main.8', 'driver_evolveflash_.1004', '<loop 460.1436>', 'hydro_.4266', 'hy_advance_.5235', '<loop 6622.5443>', 'hy_computefluxes_.5445', 'hy_getriemannstate_.6548', '<loop 6882.6621>', '<loop 6884.6623>', 'hy_datareconstonestep_.6701', '<loop 6698.7398>', 'hy_datareconstructnormaldir_ppm_.7400')",7615040.0,0.389404,0.981026
"('<program root>.6', 'main.8', 'driver_evolveflash_.1004', '<loop 460.1436>', 'hydro_.4266', 'hy_advance_.5235', '<loop 6622.5443>', 'hy_computefluxes_.5445', 'hy_getriemannstate_.6548', '<loop 6882.6621>', '<loop 6884.6623>', 'hy_datareconstonestep_.6701', '<loop 6698.7398>', 'hy_datareconstructnormaldir_ppm_.7400', 'hy_upwindtransverseflux_.7411')",1165760.0,0.059613,0.153087
"('<program root>.6', 'main.8', 'driver_evolveflash_.1004', '<loop 460.1436>', 'hydro_.4266', 'hy_advance_.5235', '<loop 6622.5443>', 'hy_computefluxes_.5445', 'hy_getriemannstate_.6548', '<loop 6882.6621>', '<loop 6884.6623>', 'hy_datareconstonestep_.6701', '<loop 6698.7398>', 'hy_datareconstructnormaldir_ppm_.7400', 'hy_upwindtransverseflux_.7411', '<loop 7190.7415>')",1013498.0,0.051826,0.869388


In [52]:
hot_paths['outlined', 'Sedov'].compact[-2:]

,CPUTIME (usec):Mean (I),CPUTIME (usec):Mean (I) ratio of total,CPUTIME (usec):Mean (I) ratio of parent
"('<program root>.6', 'main.8', 'driver_evolveflash_.859', '<loop 460.1273>', 'hydro_.4166', 'hy_advance_.5116', '<loop 6628.5311>', 'hy_computefluxes_.5313', 'hy_getriemannstate_.6438', '<loop 6888.6519>', '<loop 6890.6521>', 'hy_datareconstonestep_.6602', '<loop 6704.7269>', 'hy_datareconstructnormaldir_ppm_.7271')",7950080.0,0.394674,0.982663
"('<program root>.6', 'main.8', 'driver_evolveflash_.859', '<loop 460.1273>', 'hydro_.4166', 'hy_advance_.5116', '<loop 6628.5311>', 'hy_computefluxes_.5313', 'hy_getriemannstate_.6438', '<loop 6888.6519>', '<loop 6890.6521>', 'hy_datareconstonestep_.6602', '<loop 6704.7269>', 'hy_datareconstructnormaldir_ppm_.7271', 'hy_upwindtransverseflux_.7282')",1521750.0,0.075546,0.191413


In [53]:
hot_paths['autoinlined', 'Sedov'].compact[-2:]

,CPUTIME (usec):Mean (I),CPUTIME (usec):Mean (I) ratio of total,CPUTIME (usec):Mean (I) ratio of parent
"('<program root>.6', 'main.8', 'driver_evolveflash_.1006', '<loop 460.1422>', 'hydro_.4494', 'hy_advance_.5477', '<loop 6624.5674>', 'hy_computefluxes_.5676', 'hy_getriemannstate_.6771', '<loop 6884.6828>', '<loop 6886.6830>', 'hy_datareconstonestep_.6911', '<loop 6700.7566>', 'hy_datareconstructnormaldir_ppm_.7568')",7436440.0,0.380932,0.981087
"('<program root>.6', 'main.8', 'driver_evolveflash_.1006', '<loop 460.1422>', 'hydro_.4494', 'hy_advance_.5477', '<loop 6624.5674>', 'hy_computefluxes_.5676', 'hy_getriemannstate_.6771', '<loop 6884.6828>', '<loop 6886.6830>', 'hy_datareconstonestep_.6911', '<loop 6700.7566>', 'hy_datareconstructnormaldir_ppm_.7568', 'hy_upwindtransverseflux_.7579')",1105940.0,0.056652,0.148719


In [54]:
times = {
    _: timing.at_paths(suffix=(re.compile('hy_upwindtransverseflux_\..*'),)).compact
    for _, timing in timings.items()}

In [69]:
comparisons = {}
for problem in problems:
    problem_times = {name: val for name, val in times.items() if name[1] == problem}
    comparisons[problem] = pd.DataFrame(
        data=[val.values.tolist()[0] for _, val in problem_times.items()], index=list(problem_times.keys()),
        columns=next(iter(problem_times.values())).columns.values)

    base = comparisons[problem].at[('base', problem), 'CPUTIME (usec):Mean (I)']
    comparisons[problem].insert(
        0, 'speedup vs base',
        [base / row['CPUTIME (usec):Mean (I)'] for _, row in comparisons[problem].iterrows()])

In [70]:
comparisons['Sedov']

,speedup vs base,CPUTIME (usec):Mean (I),CPUTIME (usec):Mean (I) ratio of total,CPUTIME (usec):Mean (I) ratio of parent
"(base, Sedov)",1.000000,1165760.0,0.059613,0.153087
"(outlined, Sedov)",0.766065,1521750.0,0.075546,0.191413
"(autoinlined, Sedov)",1.054090,1105940.0,0.056652,0.148719


In [71]:
comparisons['Sod']

,speedup vs base,CPUTIME (usec):Mean (I),CPUTIME (usec):Mean (I) ratio of total,CPUTIME (usec):Mean (I) ratio of parent
"(base, Sod)",1.000000,945516.0,0.081821,0.293096
"(outlined, Sod)",0.783505,1206778.0,0.097547,0.333384
"(autoinlined, Sod)",1.005937,939936.0,0.081339,0.286377


# scaling

experiment 3

In [ ]:
logging.getLogger('hpctoolkit_dataframe').setLevel(logging.INFO)

In [11]:
%%time

timings = {}
hot_paths = {}

for (branch, nickname), problem, mpi_proc in itertools.product(branch_nicknames.items(), problems, mpi_processes):
    test_name = test_name_template.format(problem, nickname, mpi_proc)
    _profile_path = profile_db_path(test_date, test_name=test_name).joinpath('experiment.xml')
    df = HPCtoolkitDataFrame(path=_profile_path, max_depth=None)
    timings[nickname, problem, mpi_proc] = df
    hot_paths[nickname, problem, mpi_proc] = df.hot_path()

CPU times: user 36.7 s, sys: 860 ms, total: 37.6 s
Wall time: 37.6 s


In [18]:
_cols = ['PAPI_TOT_CYC:Mean (I)', 'CPUTIME (usec):Mean (I)', 'CPUTIME (usec):Mean:num-src (I)']

In [19]:
hot_paths['base', 'Sod', 1][_cols][-3:]

,PAPI_TOT_CYC:Mean (I),CPUTIME (usec):Mean (I),CPUTIME (usec):Mean:num-src (I)
"('<program root>.6', 'main.8', 'driver_evolveflash_.641', '<loop 458.710>', 'hydro_.3051', 'hy_advance_.4517', '<loop 6582.4825>', 'hy_computefluxes_.4827', 'hy_getriemannstate_.6992', '<loop 6842.7121>', '<loop 6844.7123>', 'hy_datareconstonestep_.7388', '<loop 6658.8226>', 'hy_datareconstructnormaldir_mh_.8280')",1.264030e+10,3948620.0,1.0
"('<program root>.6', 'main.8', 'driver_evolveflash_.641', '<loop 458.710>', 'hydro_.3051', 'hy_advance_.4517', '<loop 6582.4825>', 'hy_computefluxes_.4827', 'hy_getriemannstate_.6992', '<loop 6842.7121>', '<loop 6844.7123>', 'hy_datareconstonestep_.7388', '<loop 6658.8226>', 'hy_datareconstructnormaldir_mh_.8280', 'hy_upwindtransverseflux_.8472')",3.791800e+09,1048990.0,1.0
"('<program root>.6', 'main.8', 'driver_evolveflash_.641', '<loop 458.710>', 'hydro_.3051', 'hy_advance_.4517', '<loop 6582.4825>', 'hy_computefluxes_.4827', 'hy_getriemannstate_.6992', '<loop 6842.7121>', '<loop 6844.7123>', 'hy_datareconstonestep_.7388', '<loop 6658.8226>', 'hy_datareconstructnormaldir_mh_.8280', 'hy_upwindtransverseflux_.8472', '<loop 7150.8528>')",3.282500e+09,857432.0,1.0


In [20]:
hot_paths['base', 'Sod', 2][_cols][-3:]

,PAPI_TOT_CYC:Mean (I),CPUTIME (usec):Mean (I),CPUTIME (usec):Mean:num-src (I)
"('<program root>.14', 'main.16', 'driver_evolveflash_.1825', '<loop 458.1999>', 'hydro_.7096', 'hy_advance_.9422', '<loop 6582.10102>', 'hy_computefluxes_.10104', 'hy_getriemannstate_.12216', '<loop 6842.12353>', '<loop 6844.12355>', 'hy_datareconstonestep_.12526', '<loop 6658.13562>', 'hy_datareconstructnormaldir_mh_.13621')",6.310650e+09,2151080.0,2.0
"('<program root>.14', 'main.16', 'driver_evolveflash_.1825', '<loop 458.1999>', 'hydro_.7096', 'hy_advance_.9422', '<loop 6582.10102>', 'hy_computefluxes_.10104', 'hy_getriemannstate_.12216', '<loop 6842.12353>', '<loop 6844.12355>', 'hy_datareconstonestep_.12526', '<loop 6658.13562>', 'hy_datareconstructnormaldir_mh_.13621', 'hy_upwindtransverseflux_.13919')",1.889900e+09,592360.0,2.0
"('<program root>.14', 'main.16', 'driver_evolveflash_.1825', '<loop 458.1999>', 'hydro_.7096', 'hy_advance_.9422', '<loop 6582.10102>', 'hy_computefluxes_.10104', 'hy_getriemannstate_.12216', '<loop 6842.12353>', '<loop 6844.12355>', 'hy_datareconstonestep_.12526', '<loop 6658.13562>', 'hy_datareconstructnormaldir_mh_.13621', 'hy_upwindtransverseflux_.13919', '<loop 7150.13980>')",1.626650e+09,516650.0,2.0


In [21]:
hot_paths['base', 'Sod', 4][_cols][-1:]

,PAPI_TOT_CYC:Mean (I),CPUTIME (usec):Mean (I),CPUTIME (usec):Mean:num-src (I)
"('<program root>.14', 'main.16', 'driver_evolveflash_.3396', '<loop 458.3681>', 'hydro_.10708', 'hy_advance_.13539', '<loop 6582.14856>', 'hy_computefluxes_.14858', 'hy_getriemannstate_.16763', '<loop 6842.16886>', '<loop 6844.16888>', 'hy_datareconstonestep_.17145', '<loop 6658.17987>', 'hy_datareconstructnormaldir_mh_.18041')",3.152425e+09,1095135.0,4.0


In [46]:
# total_cycles = {}
column = 'PAPI_TOT_CYC:Mean (I)'
#column = 'CPUTIME (usec):Mean (I)'
for (branch, nickname), problem in itertools.product(branch_nicknames.items(), problems):
    # print(_timing['PAPI_TOT_CYC:Mean (I)'] * mpi_proc, mpi_proc)
    #total_cycles[nickname, problem, mpi_proc] = _timing['PAPI_TOT_CYC:Mean (I)']
    print(branch, nickname)
    base_mpi_proc = mpi_processes[0]
    print('scaling efficiency vs', base_mpi_proc, 'MPI rank(s)')
    base_value = timings[nickname, problem, base_mpi_proc].loc['()'][column] * base_mpi_proc
    #print(base_value)
    for mpi_proc in mpi_processes[1:]:
        value = timings[nickname, problem, mpi_proc].loc['()'][column] * mpi_proc
        #print(value)
        loss = (value - base_value) / value
        eff = 1.0 - loss
        print('at', mpi_proc, 'MPI ranks:', eff, 'i.e. loss:', loss)

transpyle_experiments base
scaling efficiency vs 1 MPI rank(s)
at 2 MPI ranks: 0.8985329456301092 i.e. loss: 0.10146705436989083
at 4 MPI ranks: 0.5713035894931454 i.e. loss: 0.4286964105068546


In [18]:
metrics_by_id = {4: 'PAPI_TOT_CYC:Sum (I)',
 5: 'PAPI_TOT_CYC:Mean (I)',
 6: 'PAPI_TOT_CYC:Mean:num-src (I)',
 7: 'PAPI_TOT_CYC:StdDev (I)',
 8: 'PAPI_TOT_CYC:StdDev:accum2 (I)',
 9: 'PAPI_TOT_CYC:StdDev:num-src (I)',
 10: 'PAPI_TOT_CYC:CfVar (I)',
 11: 'PAPI_TOT_CYC:CfVar:accum2 (I)',
 12: 'PAPI_TOT_CYC:CfVar:num-src (I)',
 13: 'PAPI_TOT_CYC:Min (I)',
 14: 'PAPI_TOT_CYC:Max (I)',
 15: 'PAPI_TOT_CYC:Sum (E)',
 16: 'PAPI_TOT_CYC:Mean (E)',
 17: 'PAPI_TOT_CYC:Mean:num-src (E)',
 18: 'PAPI_TOT_CYC:StdDev (E)',
 19: 'PAPI_TOT_CYC:StdDev:accum2 (E)',
 20: 'PAPI_TOT_CYC:StdDev:num-src (E)',
 21: 'PAPI_TOT_CYC:CfVar (E)',
 22: 'PAPI_TOT_CYC:CfVar:accum2 (E)',
 23: 'PAPI_TOT_CYC:CfVar:num-src (E)',
 24: 'PAPI_TOT_CYC:Min (E)',
 25: 'PAPI_TOT_CYC:Max (E)',
 26: 'CPUTIME (usec):Sum (I)',
 27: 'CPUTIME (usec):Mean (I)',
 28: 'CPUTIME (usec):Mean:num-src (I)',
 29: 'CPUTIME (usec):StdDev (I)',
 30: 'CPUTIME (usec):StdDev:accum2 (I)',
 31: 'CPUTIME (usec):StdDev:num-src (I)',
 32: 'CPUTIME (usec):CfVar (I)',
 33: 'CPUTIME (usec):CfVar:accum2 (I)',
 34: 'CPUTIME (usec):CfVar:num-src (I)',
 35: 'CPUTIME (usec):Min (I)',
 36: 'CPUTIME (usec):Max (I)',
 37: 'CPUTIME (usec):Sum (E)',
 38: 'CPUTIME (usec):Mean (E)',
 39: 'CPUTIME (usec):Mean:num-src (E)',
 40: 'CPUTIME (usec):StdDev (E)',
 41: 'CPUTIME (usec):StdDev:accum2 (E)',
 42: 'CPUTIME (usec):StdDev:num-src (E)',
 43: 'CPUTIME (usec):CfVar (E)',
 44: 'CPUTIME (usec):CfVar:accum2 (E)',
 45: 'CPUTIME (usec):CfVar:num-src (E)',
 46: 'CPUTIME (usec):Min (E)',
 47: 'CPUTIME (usec):Max (E)'}

In [19]:
data = {'PAPI_TOT_CYC:Sum (I)': 40943900000.0, 'PAPI_TOT_CYC:Mean (I)': 40943900000.0, 'PAPI_TOT_CYC:Mean:num-src (I)': 1.0, 'PAPI_TOT_CYC:StdDev (I)': 40943900000.0, 'PAPI_TOT_CYC:StdDev:accum2 (I)': 1.6764e+21, 'PAPI_TOT_CYC:StdDev:num-src (I)': 1.0, 'PAPI_TOT_CYC:CfVar (I)': 40943900000.0, 'PAPI_TOT_CYC:CfVar:accum2 (I)': 1.6764e+21, 'PAPI_TOT_CYC:CfVar:num-src (I)': 1.0, 'PAPI_TOT_CYC:Min (I)': 40943900000.0, 'PAPI_TOT_CYC:Max (I)': 40943900000.0, 'PAPI_TOT_CYC:Mean:num-src (E)': 1.0, 'PAPI_TOT_CYC:StdDev:num-src (E)': 1.0, 'PAPI_TOT_CYC:CfVar:num-src (E)': 1.0, 'PAPI_TOT_CYC:Min (E)': 2.22507e-308, 'CPUTIME (usec):Sum (I)': 14399400.0, 'CPUTIME (usec):Mean (I)': 14399400.0, 'CPUTIME (usec):Mean:num-src (I)': 1.0, 'CPUTIME (usec):StdDev (I)': 14399400.0, 'CPUTIME (usec):StdDev:accum2 (I)': 207342000000000.0, 'CPUTIME (usec):StdDev:num-src (I)': 1.0, 'CPUTIME (usec):CfVar (I)': 14399400.0, 'CPUTIME (usec):CfVar:accum2 (I)': 207342000000000.0, 'CPUTIME (usec):CfVar:num-src (I)': 1.0, 'CPUTIME (usec):Min (I)': 14399400.0, 'CPUTIME (usec):Max (I)': 14399400.0, 'CPUTIME (usec):Mean:num-src (E)': 1.0, 'CPUTIME (usec):StdDev:num-src (E)': 1.0, 'CPUTIME (usec):CfVar:num-src (E)': 1.0, 'CPUTIME (usec):Min (E)': 2.22507e-308, 'location': ()}

In [31]:
data

{'PAPI_TOT_CYC:Sum (I)': 40943900000.0,
 'PAPI_TOT_CYC:Mean (I)': 40943900000.0,
 'PAPI_TOT_CYC:Mean:num-src (I)': 1.0,
 'PAPI_TOT_CYC:StdDev (I)': 40943900000.0,
 'PAPI_TOT_CYC:StdDev:accum2 (I)': 1.6764e+21,
 'PAPI_TOT_CYC:StdDev:num-src (I)': 1.0,
 'PAPI_TOT_CYC:CfVar (I)': 40943900000.0,
 'PAPI_TOT_CYC:CfVar:accum2 (I)': 1.6764e+21,
 'PAPI_TOT_CYC:CfVar:num-src (I)': 1.0,
 'PAPI_TOT_CYC:Min (I)': 40943900000.0,
 'PAPI_TOT_CYC:Max (I)': 40943900000.0,
 'PAPI_TOT_CYC:Mean:num-src (E)': 1.0,
 'PAPI_TOT_CYC:StdDev:num-src (E)': 1.0,
 'PAPI_TOT_CYC:CfVar:num-src (E)': 1.0,
 'PAPI_TOT_CYC:Min (E)': 2.22507e-308,
 'CPUTIME (usec):Sum (I)': 14399400.0,
 'CPUTIME (usec):Mean (I)': 14399400.0,
 'CPUTIME (usec):Mean:num-src (I)': 1.0,
 'CPUTIME (usec):StdDev (I)': 14399400.0,
 'CPUTIME (usec):StdDev:accum2 (I)': 207342000000000.0,
 'CPUTIME (usec):StdDev:num-src (I)': 1.0,
 'CPUTIME (usec):CfVar (I)': 14399400.0,
 'CPUTIME (usec):CfVar:accum2 (I)': 207342000000000.0,
 'CPUTIME (usec):CfVar:nu

In [23]:
from math import sqrt

In [22]:
sqrt(
    (data.get(metrics_by_id[8]) / data.get(metrics_by_id[9]))
    - pow(data.get(metrics_by_id[7]) / data.get(metrics_by_id[9]), 2))

ValueError: math domain error

In [29]:
_ = (data.get(metrics_by_id[8]) / data.get(metrics_by_id[9])) - pow(data.get(metrics_by_id[7]) / data.get(metrics_by_id[9]), 2)

In [30]:
sqrt(_)

ValueError: math domain error

## SNIa scaling

experiment 4

In [15]:
%%time

timings = {}
hot_paths = {}

for (branch, nickname), (problem, options), mpi_proc in itertools.product(
        branch_nicknames.items(), problems, mpi_processes):
    test_name = test_name_template.format(problem, nickname, mpi_proc)
    _profile_path = profile_db_path(test_date, test_name=test_name).joinpath('experiment.xml')
    df = HPCtoolkitDataFrame(path=_profile_path)
    timings[nickname, problem, mpi_proc] = df
    hot_paths[nickname, problem, mpi_proc] = df.hot_path()

CPU times: user 6.43 s, sys: 80.9 ms, total: 6.51 s
Wall time: 6.57 s


In [ ]:
hot_paths['base', 'SNIa_DoubleDetonation', 0][:]

In [22]:
timings['base', 'SNIa_DoubleDetonation', 0] \
    .at_paths(prefix=('<program root>.10', 'main.12', 'driver_evolveflash_.698', '<loop 458.748>',
                      'hydro_.3210', 'hy_advance_.4836')) \
    .at_depth(7).compact

,CPUTIME (usec):Mean (I),CPUTIME (usec):Mean (I) ratio of total,CPUTIME (usec):Mean (I) ratio of parent
"('<program root>.10', 'main.12', 'driver_evolveflash_.698', '<loop 458.748>', 'hydro_.3210', 'hy_advance_.4836', 'grid_putfluxdata_.4838')",123758.0,0.008609,0.012841
"('<program root>.10', 'main.12', 'driver_evolveflash_.698', '<loop 458.748>', 'hydro_.3210', 'hy_advance_.4836', 'timers_startstring_.4938')",3991.0,0.000278,0.000414
"('<program root>.10', 'main.12', 'driver_evolveflash_.698', '<loop 458.748>', 'hydro_.3210', 'hy_advance_.4836', 'grid_conservefluxes_.4946')",79471.0,0.005528,0.008246
"('<program root>.10', 'main.12', 'driver_evolveflash_.698', '<loop 458.748>', 'hydro_.3210', 'hy_advance_.4836', 'timers_stopstring_.5130')",NaN,NaN,NaN
"('<program root>.10', 'main.12', 'driver_evolveflash_.698', '<loop 458.748>', 'hydro_.3210', 'hy_advance_.4836', '<loop 6582.5132>')",9246560.0,0.643248,0.959441
"('<program root>.10', 'main.12', 'driver_evolveflash_.698', '<loop 458.748>', 'hydro_.3210', 'hy_advance_.4836', '<loop 6584.9815>')",183670.0,0.012777,0.019058


### Profiling data analysis itself

In [9]:
%%time

import line_profiler
profiler = line_profiler.LineProfiler()
profiler.add_module(hpctoolkit_dataframe)
profiler.enable()

_profile_path = profile_db_path(
    datetime.datetime(2018, 9, 5, 12, 5, 19, 934255), test_name='subset_Sedov_base_a').joinpath('experiment.xml')
df = HPCtoolkitDataFrame(path=_profile_path, max_depth=None)

profiler.disable()

CPU times: user 7.57 s, sys: 66.9 ms, total: 7.64 s
Wall time: 7.63 s


In [10]:
%%time

profiler.enable()

hot_path = df.hot_path(threshold=0.01)

profiler.disable()

CPU times: user 4 s, sys: 3.48 ms, total: 4 s
Wall time: 3.99 s


In [11]:
%%time

profiler.enable()

df.compact
df.basic_i

profiler.disable()

CPU times: user 3.73 ms, sys: 0 ns, total: 3.73 ms
Wall time: 2.93 ms


In [12]:
profiler.print_stats()

Timer unit: 1e-06 s

Total time: 0.241474 s
File: /nfs2/mbysiek/Projects/docker-transpyle-flash/hpctoolkit_dataframe.py
Function: _read_xml at line 16

Line #      Hits         Time  Per Hit   % Time  Line Contents
    16                                           def _read_xml(path: pathlib.Path) -> ET.ElementTree:
    17         1       1283.0   1283.0      0.5      with path.open() as xml_file:
    18         1     240189.0 240189.0     99.5          xml_data = ET.parse(xml_file)
    19         1          2.0      2.0      0.0      return xml_data

Total time: 4.8e-05 s
File: /nfs2/mbysiek/Projects/docker-transpyle-flash/hpctoolkit_dataframe.py
Function: _metrics_formula_sub_predicate at line 22

Line #      Hits         Time  Per Hit   % Time  Line Contents
    22                                           def _metrics_formula_sub_predicate(match: t.Match) -> str:
    23        36         48.0      1.3    100.0      return 'data.get(self._metrics_by_id[{}])'.format(match.group()[1:])

In [13]:
hot_path.compact

,CPUTIME (usec):Mean (I),CPUTIME (usec):Mean (I) ratio of total,CPUTIME (usec):Mean (I) ratio of parent
(),19555620.0,1.000000,1.000000
"('<program root>.6',)",19542820.0,0.999345,0.999345
"('<program root>.6', 'main.8')",19542820.0,0.999345,1.000000
"('<program root>.6', 'main.8', 'driver_evolveflash_.1004')",19218840.0,0.982778,0.983422
"('<program root>.6', 'main.8', 'driver_evolveflash_.1004', '<loop 460.1436>')",19180260.0,0.980806,0.997993
"('<program root>.6', 'main.8', 'driver_evolveflash_.1004', '<loop 460.1436>', 'hydro_.4266')",16446600.0,0.841017,0.857475
"('<program root>.6', 'main.8', 'driver_evolveflash_.1004', '<loop 460.1436>', 'hydro_.4266', 'hy_advance_.5235')",12673020.0,0.648050,0.770556
"('<program root>.6', 'main.8', 'driver_evolveflash_.1004', '<loop 460.1436>', 'hydro_.4266', 'hy_advance_.5235', '<loop 6622.5443>')",12300100.0,0.628980,0.970574
"('<program root>.6', 'main.8', 'driver_evolveflash_.1004', '<loop 460.1436>', 'hydro_.4266', 'hy_advance_.5235', '<loop 6622.5443>', 'hy_computefluxes_.5445')",12288460.0,0.628385,0.999054
"('<program root>.6', 'main.8', 'driver_evolveflash_.1004', '<loop 460.1436>', 'hydro_.4266', 'hy_advance_.5235', '<loop 6622.5443>', 'hy_computefluxes_.5445', 'hy_getriemannstate_.6548')",9489420.0,0.485253,0.772222


### Profiling data access

#### 1

`DataFrame.loc[[str(...)]]` vs `DataFrame[DataFrame.col == ...]` vs `DataFrame[DataFrame['col'] == ...]`

assuming that `DataFrame` is indexed by stringified column `col`

In [18]:
%%timeit

df.loc[[str(())]]

544 µs ± 20.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [19]:
type(df.loc[[str(())]])

hpctoolkit_dataframe.HPCtoolkitDataFrame

In [20]:
%%timeit

df[df.location == ()]

804 µs ± 69.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [21]:
type(df[df.location == ()])

hpctoolkit_dataframe.HPCtoolkitDataFrame

In [22]:
%%timeit

df[df['location'] == ()]

739 µs ± 12.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


#### 2

`Series.at[...]` vs `Series.get(...)` vs `Series[...]`

In [23]:
filtered = df.loc['()']

In [24]:
type(filtered)

pandas.core.series.Series

In [25]:
%%timeit

df.loc['()']

133 µs ± 1.29 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [26]:
%%timeit

filtered.at['location']

8.31 µs ± 59.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [27]:
type(filtered.at['location'])

tuple

In [28]:
%%timeit

filtered.get('location')

9.68 µs ± 115 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [29]:
type(filtered.get('location'))

tuple

In [30]:
%%timeit

filtered['location']

9.48 µs ± 34.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [31]:
type(filtered['location'])

tuple

#### 3

`DataFrame.at['index', 'col']` vs `DataFrame.get('col').item()` vs `DataFrame['col'].item()`

assuming that `Dataframe` contains only one row (indexed as `'index'`)

In [32]:
filtered = df.loc[['()']]

In [33]:
type(filtered)

hpctoolkit_dataframe.HPCtoolkitDataFrame

In [34]:
%%timeit

df.loc[['()']]

526 µs ± 5.13 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [35]:
%%timeit

filtered.at['()', 'location']

3.92 µs ± 40.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [36]:
type(filtered.at['()', 'location'])

tuple

In [37]:
%%timeit

filtered.get('location').item()

2.56 µs ± 49.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [38]:
type(filtered.get('location').item())

tuple

In [39]:
%%timeit

filtered['location'].item()

2.38 µs ± 33.3 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [40]:
type(filtered['location'].item())

tuple